In [1]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd

import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


[(6+0j), (6+0j), (2.4+0j), (2.5999999999999996+1.637849848627056e-17j), (2.287500258169+0j), (2.4+2.2311252215006176e-17j), (2.4-2.576281494458285e-17j), (2.3000000000000003-4.125283626344832e-18j), (2.587500258169+0j)]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:239: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [2]:
#Class definitions
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def solution(self):
        solution = fsolve(Lagrange_eqs, self.initial_guess)
        return solution
    @property
    def res_list(self):
        res_list = Lagrange_eqs(self.solution)
        return res_list
    @property
    def res(self):
        res = np.sum(np.abs(self.res_list))
        return res
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.solution
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                # POVM_normalized_9d[i+2].append(part[i])
                POVM_normalized_9d[i+2].append((1/np.sqrt(6))*(part[i]))        # normalizing coming from the fixing of the lagrangian eqns
        return POVM_normalized_9d


class POVM_relations:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
        self.inner_products = {}
    def dot_product(self, i, j):
        if i < 0 or j < 0 or i > 8 or j > 8:
            raise ValueError('Bad index, must be integer 0 to 8')
        return np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
    def all(self, clean = True, threshold = 1e-12, absolutes = True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        for i in range(9):
            for j in range(i, 9):
                key = f'{i+1}{j+1}'
                value = self.dot_product(i, j)
                self.inner_products[key] = value
        #cleaning the dictionary
        rel = self.inner_products
        for key in rel:
            if absolutes == True:
                rel[key] = abs(rel[key])
            if abs(rel[key]) < threshold:
                rel[key] = 0
        return rel


In [4]:
# creating five lists POVMs
w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = [[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]             # unnormalized POVM direction vectors
# POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors
POVM_vec = (1/np.sqrt(2))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors

# Manually calculated c4j and c5j values, see the notebook for the calculations
c4j_list = [2,.5,.5,.5, (-.25-.25j*np.sqrt(3)), (-.25+.25j*np.sqrt(3)), .5, (-.25+.25j*np.sqrt(3)), (-.25-.25j*np.sqrt(3))]           # fourth elements, not normalized yet. See sic.py for calcs
c5j_list = [0, np.sqrt(15)/2 , (1/np.sqrt(15))*(3/2), (1/np.sqrt(15))*(2*w**2-.5) ,-(1/np.sqrt(15))*(2+(5/2)*w**2), (1/np.sqrt(15))*(2-w/2), (1/np.sqrt(15))*(2*w-.5), -(1/np.sqrt(15))*(2+(5/2)*w), (1/np.sqrt(15))*(2-.5*w**2)]

POVM_unnormalized_5d = []
for i in range(len(POVM_unnormalized)):
	# Directly append c4j_list[i] and c5j_list[i] to the copies of lists in POVM_unnormalized
	vec_5d_i = POVM_unnormalized[i] + [c4j_list[i], c5j_list[i]]
	POVM_unnormalized_5d.append(vec_5d_i)
POVM_normalized_5d = ((1/(6**.5))*(np.array(POVM_unnormalized_5d))).tolist()       # normalized 5-lists POVM vectors

delta_five_fixed = [ np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(2,9)]     # print('delta_five_fixed = ', delta_five_fixed)



In [7]:
# the main objective function containing the Lagrange equations
def Lagrange_eqs(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99,  y3, y4, y5, y6, y7, y8, y9 = vars
    # global delta_five
    # delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five
    global delta_five_new
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five_fixed

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        (1/6)*(np.conj(C63)*y3*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64)*y4*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65)*y5*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9*(1/6)),
        # Continue with C73 to C79
        # Derivatives with respect to C73 to C79
        (1/6)*(np.conj(C73)*y3*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74)*y4*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75)*y5*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9*(1/6)),
        # Derivatives with respect to C83 to C89
        (1/6)*(np.conj(C83)*y3*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84)*y4*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85)*y5*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9*(1/6)),
        # Derivatives with respect to C93 to C99
        (1/6)*(np.conj(C93)*y3*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94)*y4*(1/6)), 
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95)*y5*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9*(1/6)),
        #Now the normalization equations
        (1/6)*(abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2) + delta_33 - 1,  
        (1/6)*(abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2) + delta_44 - 1,
        (1/6)*(abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2) + delta_55 - 1,  
        (1/6)*(abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2) + delta_66 - 1,
        (1/6)*(abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2) + delta_77 - 1,  
        (1/6)*(abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2) + delta_88 - 1,
        (1/6)*(abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2) + delta_99 - 1
    ]
    assert len(equations) == 35
    return equations


In [8]:
# ig below for np.seed = 2918 
# initial_guess =[0.98030858, 0.35567294, 0.99141023, 0.47111957, 0.34998446, 0.43261049, 0.30442865, 0.21036997, 0.39912775, 0.24321787, 0.91072068, 0.59991195, 0.39894914, 0.9238744 , 0.27633149, 0.9263308 , 0.02004467, 0.09815871, 0.03103966, 0.9009708 , 0.60346855, 0.30131413, 0.22667874, 0.99340599, 0.71874642, 0.71156703, 0.8234102 , 0.80459364, 0.77652217, 0.75734249, 0.06087287, 0.93906535, 0.92258927, 0.22224891, 0.0029041 ] 
initial_guess =[0.13645285, 0.84315108 ,0.12490685 ,0.10271676 ,0.27940578 ,0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531,  0.71094717, 0.34003065, 0.7689673,  0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829 ]
print("Initial guess:", initial_guess)
# initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5   # Initial guess for the variables (Cij real and imaginary parts, y)
solution = fsolve(Lagrange_eqs, initial_guess)
residuals = Lagrange_eqs(solution)                   # residuals and residues sum
residuals_sum = np.sum(np.abs(residuals))

print("Solution to the system:", solution)
print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)

Initial guess: [0.13645285, 0.84315108, 0.12490685, 0.10271676, 0.27940578, 0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531, 0.71094717, 0.34003065, 0.7689673, 0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829]
Solution to the system: [-1.50824653e+00  1.46575216e+00  1.46575216e+00 -1.46575216e+00
 -1.46575216e+00  1.46575216e+00  1.46575216e+00  3.05920972e-01
 -2.97301745e-01 -2.97301745e-01  2.97301745e-01  2.97301744e-01
 -2.97301744e-01 -2.97301744e-01  9.70985599e-01 -9.43628386e-01
 -9.43628387e-01  9.43628387e-01  9.43628387e-01 -9.43628387e-01
 -9.43628386e-01  5.37393468e-01 -5.22252577e-01 -5.22252578e-01
  5.22252578e-01  5.22252578e-01 -5.22252578e-01 -5.22252578e-01
 -2.65178181e-25  6.17394907e+00  1.73949065e-01  5.82605093e+00
 -1.73949065e-01  6.1739